In [2]:
import pandas as pd
import os
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier
import lightgbm as lgb

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import cross_validate
# from sklearn.inspection import permutation_importance

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [9]:
data_path = '/Users/sola/Downloads/open/'
train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [10]:
wrong = []
for i in range(1,2876):
    ch = 'X_'+str(i)
    if train[ch].mean() == 0:
        wrong.append(ch)

print(wrong)
print(len(wrong))

['X_3', 'X_6', 'X_14', 'X_70', 'X_74', 'X_75', 'X_76', 'X_77', 'X_78', 'X_79', 'X_82', 'X_83', 'X_84', 'X_85', 'X_142', 'X_190', 'X_191', 'X_192', 'X_193', 'X_194', 'X_195', 'X_232', 'X_234', 'X_235', 'X_259', 'X_260', 'X_315', 'X_319', 'X_322', 'X_323', 'X_324', 'X_327', 'X_328', 'X_329', 'X_330', 'X_375', 'X_376', 'X_377', 'X_378', 'X_427', 'X_466', 'X_467', 'X_505', 'X_508', 'X_558', 'X_561', 'X_567', 'X_583', 'X_617', 'X_620', 'X_631', 'X_633', 'X_634', 'X_636', 'X_638', 'X_639', 'X_640', 'X_641', 'X_642', 'X_672', 'X_673', 'X_676', 'X_691', 'X_692', 'X_695', 'X_715', 'X_729', 'X_732', 'X_743', 'X_749', 'X_759', 'X_760', 'X_761', 'X_764', 'X_776', 'X_777', 'X_778', 'X_836', 'X_843', 'X_844', 'X_849', 'X_859', 'X_886', 'X_887', 'X_888', 'X_889', 'X_992', 'X_1020', 'X_1021', 'X_1022', 'X_1023', 'X_1024', 'X_1025', 'X_1070', 'X_1092', 'X_1119', 'X_1137', 'X_1146', 'X_1206', 'X_1216', 'X_1219', 'X_1248', 'X_1249', 'X_1250', 'X_1251', 'X_1252', 'X_1253', 'X_1255', 'X_1293', 'X_1298', 'X

In [11]:
train = train.drop(wrong, axis=1)
test = test.drop(wrong, axis=1)

# train = train.drop(['TIMESTAMP','PRODUCT_ID'], axis=1)
# test = test.drop(['TIMESTAMP','PRODUCT_ID'], axis=1)

In [13]:
#해보자!

tt = train[train['PRODUCT_CODE']=="T_31"]

ttt = tt.dropna(how="any", axis="columns")
cols = ttt.columns[6:]
for col in cols:
    if len(ttt[col].unique()) == 1:
        ttt = ttt.drop([col], axis=1)
t_col = ttt.columns[6:]
ttt
t_col

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_4,X_5,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,TRAIN_003,2,0.537325,2022-06-13 5:39,T050307,A_31,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,TRAIN_004,1,0.531590,2022-06-13 5:47,T050304,A_31,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,TRAIN_593,1,0.526546,2022-09-08 14:30,T100306,T_31,2.0,95.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,TRAIN_594,0,0.524022,2022-09-08 22:38,T050304,A_31,NaN,NaN,NaN,NaN,...,49.47,53.07,50.89,55.10,66.49,1.0,NaN,NaN,NaN,NaN
595,TRAIN_595,0,0.521289,2022-09-08 22:47,T050304,A_31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
596,TRAIN_596,1,0.531375,2022-09-08 14:38,T100304,O_31,40.0,94.0,45.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
lines = ['T050304', 'T050307', 'T100304', 'T100306', 'T010306', 'T010305']


def seperate_code():
    # data_path = '/Users/sola/Downloads/open/'
    output_dir = "Input/"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    df = train

    for line in lines:
        df_line = df[df["LINE"] == line]
        df_line = df_line.dropna(how="any", axis="columns")

        # 모든 값이 동일한 열 제거
        cols = df_line.columns[6:]
        for col in cols:
            if len(df_line[col].unique()) == 1:
                df_line = df_line.drop([col], axis=1)

        df_line.to_csv(os.path.join(output_dir, f"{line}.csv"), index=False)


seperate_code()

,X_1,X_2,X_4,X_5,X_7,X_8,X_9,X_10,X_11,X_12,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,2.0,95.0,45.0,10.0,50.0,10.0,52.0,2.0,468.9,473.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49.47,53.07,50.89,55.10,66.49,1.0,NaN,NaN,NaN,NaN
595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
596,40.0,94.0,45.0,11.0,45.0,10.0,31.0,2.0,505.8,510.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [140]:
def find_duplicate_col():
    # data_path = '/Users/sola/Downloads/open/'
    df = train
    df = df.round(0)

    drop_cols = []
    for line in lines:
        df_line = df[df["LINE"] == line]
        df_line = df_line.dropna(how="any", axis="columns")

        df_line = df_line.loc[:, ~df.T.duplicated()]

        cols = df_line.columns[6:]
        for col in cols:
            if len(df_line[col].unique()) == 1:
                df_line = df_line.drop([col], axis=1)

        drop_cols.append(df_line.columns)

    return drop_cols


find_duplicate_col()

[Index(['PRODUCT_ID', 'Y_Class', 'Y_Quality', 'TIMESTAMP', 'LINE',
        'PRODUCT_CODE', 'X_128', 'X_129', 'X_136', 'X_137',
        ...
        'X_2796', 'X_2797', 'X_2798', 'X_2799', 'X_2800', 'X_2801', 'X_2837',
        'X_2839', 'X_2840', 'X_2841'],
       dtype='object', length=301),
 Index(['PRODUCT_ID', 'Y_Class', 'Y_Quality', 'TIMESTAMP', 'LINE',
        'PRODUCT_CODE', 'X_130', 'X_136', 'X_137', 'X_138',
        ...
        'X_2796', 'X_2797', 'X_2798', 'X_2799', 'X_2800', 'X_2801', 'X_2837',
        'X_2839', 'X_2840', 'X_2841'],
       dtype='object', length=630),
 Index(['PRODUCT_ID', 'Y_Class', 'Y_Quality', 'TIMESTAMP', 'LINE',
        'PRODUCT_CODE', 'X_1', 'X_2', 'X_5', 'X_11',
        ...
        'X_773', 'X_774', 'X_775', 'X_883', 'X_884', 'X_885', 'X_898', 'X_899',
        'X_900', 'X_901'],
       dtype='object', length=120),
 Index(['PRODUCT_ID', 'Y_Class', 'Y_Quality', 'TIMESTAMP', 'LINE',
        'PRODUCT_CODE', 'X_1', 'X_2', 'X_7', 'X_8',
        ...
        'X

In [141]:
def make_col():
    col_list = []
    for l, line in enumerate(lines):
        df = pd.read_csv(f"Input/{line}.csv")

        X = df[find_duplicate_col()[l]].drop(["Y_Class", "Y_Quality"], axis=1).select_dtypes(exclude=['object'])

        X.to_csv(f"Input/{line}.csv", index=False)
        col_list.append(X.columns)
    print(col_list[3])
    # Code : A
    A_col = list(set(col_list[0]) & set(col_list[1]) & set(col_list[4]) & set(col_list[5]))
    # Code : T, O
    OT_col = list(set(col_list[2]) & set(col_list[3]))

    use_col = A_col + OT_col

    return use_col, A_col, OT_col


make_col()

Index(['X_1', 'X_2', 'X_7', 'X_8', 'X_11', 'X_12', 'X_13', 'X_15', 'X_16',
       'X_18',
       ...
       'X_897', 'X_898', 'X_899', 'X_900', 'X_901', 'X_902', 'X_903', 'X_904',
       'X_905', 'X_932'],
      dtype='object', length=341)


(['X_1548',
  'X_1226',
  'X_1227',
  'X_1102',
  'X_1121',
  'X_1099',
  'X_1213',
  'X_1547',
  'X_1124',
  'X_1156',
  'X_2046',
  'X_1197',
  'X_1091',
  'X_1554',
  'X_1208',
  'X_1224',
  'X_1108',
  'X_1088',
  'X_1329',
  'X_1002',
  'X_1101',
  'X_956',
  'X_1106',
  'X_2050',
  'X_1128',
  'X_1528',
  'X_1549',
  'X_1545',
  'X_1118',
  'X_1220',
  'X_1521',
  'X_1218',
  'X_1139',
  'X_1138',
  'X_2048',
  'X_1195',
  'X_1541',
  'X_1084',
  'X_1136',
  'X_1221',
  'X_1203',
  'X_1133',
  'X_1112',
  'X_1134',
  'X_1152',
  'X_1116',
  'X_1202',
  'X_1110',
  'X_2051',
  'X_1148',
  'X_1122',
  'X_1527',
  'X_1544',
  'X_1542',
  'X_1196',
  'X_2047',
  'X_1539',
  'X_1095',
  'X_1089',
  'X_1543',
  'X_1109',
  'X_1207',
  'X_1081',
  'X_1117',
  'X_955',
  'X_1151',
  'X_1204',
  'X_1079',
  'X_1193',
  'X_1551',
  'X_1107',
  'X_1086',
  'X_1143',
  'X_1223',
  'X_1097',
  'X_1077',
  'X_1078',
  'X_1215',
  'X_1140',
  'X_1153',
  'X_1125',
  'X_1546',
  'X_1194',
  'X_1

In [142]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier


def data_modeling_A():
    data_path = '/Users/sola/Downloads/open/'
    df = pd.read_csv(data_path + 'train.csv')

    # data_A = df[df["PRODUCT_CODE"] == "A_31"]

    # data_A = data_A.fillna(0)
    data_A = train[train["PRODUCT_CODE"] == "A_31"]
    A_y = data_A["Y_Class"]
    A_X = data_A.drop(["Y_Class", "Y_Quality"], axis=1).select_dtypes(exclude=['object'])

    X_train, X_test, y_train, y_test = train_test_split(A_X, A_y, test_size=0.3, random_state=42)

    model_A = lgb.LGBMClassifier(random_state=42)
    model_A.fit(X_train, y_train)
    prediction = model_A.predict(X_test)

    #특성중요도 찾기
    feat_labels = X_train.columns
    # result = model_A.feature_importances_
    # print(result.importances_mean)
    importances = model_A.feature_importances_
    # argsort : 리스트 인덱스의 정렬됐을 때의 인덱스 값 반환, [::-1] : 뒤집기
    indices = np.argsort(importances)[::-1]
    a = []
    for i in range(500):
        a.append(feat_labels[indices[i]])
        # print(a)

    # a=[]
    # for i in range(100):
    #     a.append(feat_labels[indices[i]])
    # print(a)

    print(confusion_matrix(prediction, y_test))
    print(f"A_정답률:{accuracy_score(prediction, y_test):.3f}")

    # print(a)
    # print(len(a))

    return model_A, A_X.columns,a


data_modeling_A()


[[11  5  1]
 [ 5 29  5]
 [ 1  2 16]]
A_정답률:0.747


(LGBMClassifier(random_state=42),
 Index(['X_1', 'X_2', 'X_4', 'X_5', 'X_7', 'X_8', 'X_9', 'X_10', 'X_11', 'X_12',
        ...
        'X_2866', 'X_2867', 'X_2868', 'X_2869', 'X_2870', 'X_2871', 'X_2872',
        'X_2873', 'X_2874', 'X_2875'],
       dtype='object', length=2676),
 ['X_318',
  'X_367',
  'X_1430',
  'X_1099',
  'X_1774',
  'X_368',
  'X_1295',
  'X_287',
  'X_1389',
  'X_189',
  'X_1053',
  'X_1743',
  'X_1804',
  'X_1162',
  'X_1205',
  'X_1197',
  'X_1560',
  'X_953',
  'X_1948',
  'X_1954',
  'X_1117',
  'X_959',
  'X_1207',
  'X_1336',
  'X_248',
  'X_1202',
  'X_286',
  'X_1263',
  'X_1949',
  'X_258',
  'X_1697',
  'X_1286',
  'X_1376',
  'X_1120',
  'X_1033',
  'X_1108',
  'X_1048',
  'X_1213',
  'X_1856',
  'X_1243',
  'X_1356',
  'X_345',
  'X_1379',
  'X_136',
  'X_1290',
  'X_1744',
  'X_1373',
  'X_999',
  'X_1058',
  'X_1542',
  'X_1390',
  'X_1030',
  'X_1612',
  'X_1109',
  'X_1665',
  'X_1013',
  'X_1833',
  'X_1643',
  'X_1629',
  'X_1155',
  'X_1359',


In [80]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


def data_modeling_Aa():
    data_path = '/Users/sola/Downloads/open/'
    df = pd.read_csv(data_path + 'train.csv')

    data_A = df[df["PRODUCT_CODE"] == "A_31"]
    # data_A = data_A.fillna(0)
    A_y = data_A["Y_Class"]
    A_X = data_A.drop(["Y_Class", "Y_Quality"], axis=1).select_dtypes(exclude=['object'])
    A_X = A_X[data_modeling_A()[2]]
    # A_X = A_X.columns

    X_train, X_test, y_train, y_test = train_test_split(A_X, A_y, test_size=0.3, random_state=42)

    model_A = lgb.LGBMClassifier(random_state=42)
    model_A.fit(X_train, y_train)
    prediction = model_A.predict(X_test)

    model_A.fit(X_train, y_train)
    prediction = model_A.predict(X_test)

    print(confusion_matrix(prediction, y_test))
    print(f"!!A_정답률:{accuracy_score(prediction, y_test):.3f}")

    return model_A, A_X.columns


data_modeling_Aa()

[[11  5  1]
 [ 5 29  5]
 [ 1  2 16]]
A_정답률:0.747
[[10  3  1]
 [ 6 29  6]
 [ 1  4 15]]
!!A_정답률:0.720


(LGBMClassifier(random_state=42),
 Index(['X_318', 'X_367', 'X_1430', 'X_1099', 'X_1774', 'X_368', 'X_1295',
        'X_287', 'X_1389', 'X_189',
        ...
        'X_870', 'X_871', 'X_873', 'X_913', 'X_874', 'X_875', 'X_876', 'X_877',
        'X_878', 'X_879'],
       dtype='object', length=500))

In [81]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier


def data_modeling_A():
    data_path = '/Users/sola/Downloads/open/'
    df = pd.read_csv(data_path + 'train.csv')

    # data_A = df[df["PRODUCT_CODE"] == "A_31"]

    # data_A = data_A.fillna(0)
    data_A = df[(df["PRODUCT_CODE"] == "T_31") | (df["PRODUCT_CODE"] == "O_31")]
    A_y = data_A["Y_Class"]
    A_X = data_A.drop(["Y_Class", "Y_Quality"], axis=1).select_dtypes(exclude=['object'])

    X_train, X_test, y_train, y_test = train_test_split(A_X, A_y, test_size=0.3, random_state=42)

    model_A = lgb.LGBMClassifier(random_state=42)
    model_A.fit(X_train, y_train)
    prediction = model_A.predict(X_test)

    #특성중요도 찾기
    feat_labels = X_train.columns
    # result = model_A.feature_importances_
    # print(result.importances_mean)
    importances = model_A.feature_importances_
    # argsort : 리스트 인덱스의 정렬됐을 때의 인덱스 값 반환, [::-1] : 뒤집기
    indices = np.argsort(importances)[::-1]
    a = []
    for i in range(500):
        a.append(feat_labels[indices[i]])
        # print(a)

    # a=[]
    # for i in range(100):
    #     a.append(feat_labels[indices[i]])
    # print(a)

    print(confusion_matrix(prediction, y_test))
    print(f"A_정답률:{accuracy_score(prediction, y_test):.3f}")

    # print(a)
    # print(len(a))

    return model_A, A_X.columns,a


data_modeling_A()


[[ 0  1  0]
 [ 8 82 13]
 [ 1  0  0]]
A_정답률:0.781


(LGBMClassifier(random_state=42),
 Index(['X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 'X_7', 'X_8', 'X_9', 'X_10',
        ...
        'X_2866', 'X_2867', 'X_2868', 'X_2869', 'X_2870', 'X_2871', 'X_2872',
        'X_2873', 'X_2874', 'X_2875'],
       dtype='object', length=2875),
 ['X_700',
  'X_454',
  'X_399',
  'X_658',
  'X_497',
  'X_496',
  'X_667',
  'X_786',
  'X_500',
  'X_401',
  'X_485',
  'X_907',
  'X_932',
  'X_899',
  'X_398',
  'X_659',
  'X_668',
  'X_790',
  'X_120',
  'X_554',
  'X_677',
  'X_536',
  'X_835',
  'X_783',
  'X_780',
  'X_614',
  'X_766',
  'X_491',
  'X_484',
  'X_763',
  'X_669',
  'X_107',
  'X_651',
  'X_792',
  'X_385',
  'X_498',
  'X_460',
  'X_769',
  'X_813',
  'X_501',
  'X_824',
  'X_438',
  'X_397',
  'X_716',
  'X_452',
  'X_613',
  'X_11',
  'X_515',
  'X_495',
  'X_453',
  'X_446',
  'X_422',
  'X_489',
  'X_662',
  'X_855',
  'X_389',
  'X_789',
  'X_775',
  'X_896',
  'X_571',
  'X_106',
  'X_757',
  'X_589',
  'X_514',
  'X_493',
  'X_89

In [156]:
from sklearn.ensemble import RandomForestClassifier

seperate_code()

# t만!!
def data_modeling_ttt():
    data_path = '/Users/sola/Downloads/open/'
    df = pd.read_csv(data_path + 'train.csv')
    # data_T = df[(df["PRODUCT_CODE"] == "T_31") | (df["PRODUCT_CODE"] == "O_31")]
    data_T = df[(df["PRODUCT_CODE"] == "T_31")]

    # lists = []
    # for a in make_col()[2]:
    #     lists.append(int(a[2:]))
    # lists.sort()
    #
    # cols = []  # 사용할 columns
    # for l in lists:
    #     cols.append(f"X_{l}")

    T_y = data_T["Y_Class"]
    T_X = data_T[t_col]
    print("!!")
    print(T_X)

    X_train, X_test, y_train, y_test = train_test_split(T_X, T_y, test_size=0.3, random_state=42)

    model_T = lgb.LGBMClassifier(random_state=42)
    model_T.fit(X_train, y_train)
    prediction = model_T.predict(X_test)

    #특성중요도 찾기
    feat_labels = X_train.columns
    importances = model_T.feature_importances_

    # argsort : 리스트 인덱스의 정렬됐을 때의 인덱스 값 반환, [::-1] : 뒤집기
    indices = np.argsort(importances)[::-1]
    a = []
    for i in range(X_train.shape[1]):
        if importances[indices[i]] > 1:
            a.append(feat_labels[indices[i]])
            print(importances[indices[i]])
    print(confusion_matrix(prediction, y_test))
    print(f"T_정답률:{accuracy_score(prediction, y_test):.3f}")
    print(a)
    return model_T, cols


data_modeling_ttt()

!!
     X_1    X_2   X_5   X_7   X_8   X_9   X_11   X_12   X_13  X_15  ...  \
22   2.0  102.0  11.0  45.0  10.0  31.0  514.4  522.1  505.8   0.0  ...   
23   2.0  102.0  11.0  45.0  10.0  31.0  514.6  522.6  506.0   0.0  ...   
25   2.0   97.0  11.0  45.0  10.0  31.0  517.4  526.0  508.9   0.0  ...   
26   2.0   95.0  10.0  54.0  10.0  52.0  477.4  482.4  470.2   0.0  ...   
29   2.0  100.0  11.0  45.0  10.0  31.0  516.5  526.0  507.5   0.0  ...   
..   ...    ...   ...   ...   ...   ...    ...    ...    ...   ...  ...   
589  1.0   94.0  10.0  50.0  10.0  52.0  465.5  470.2  456.1   0.0  ...   
590  1.0   89.0  10.0  50.0  10.0  52.0  466.0  470.5  458.0   0.0  ...   
591  1.0   87.0  10.0  50.0  10.0  52.0  465.9  470.5  456.8   0.0  ...   
592  2.0   98.0  10.0  45.0  10.0  31.0  505.5  510.6  498.8   0.0  ...   
593  2.0   95.0  10.0  50.0  10.0  52.0  468.9  473.9  459.2   0.0  ...   

     X_774  X_775  X_882     X_883  X_884  X_885  X_898     X_899  X_900  \
22   458.0  449.0   

(LGBMClassifier(random_state=42),
 Index(['X_1', 'X_2', 'X_4', 'X_5', 'X_7', 'X_8', 'X_9', 'X_10', 'X_11', 'X_12',
        ...
        'X_774', 'X_775', 'X_882', 'X_883', 'X_884', 'X_885', 'X_898', 'X_899',
        'X_900', 'X_901'],
       dtype='object', length=203))

In [164]:
pred_O=[]
for i in range(4):
    pred_O.append(0)

In [165]:
def predict_testset():
    data_path = '/Users/sola/Downloads/open/'
    df = pd.read_csv(data_path + 'test.csv')

    data_A = data_modeling_A()
    data_T = data_modeling_ttt()

    # A_testset
    df_A = df[df["PRODUCT_CODE"] == "A_31"]

    df_test_A = pd.DataFrame()
    A_cols = data_A[1]

    for c, col in enumerate(A_cols):
        df_test_A[col] = df_A[col]

    # T_testset
    df_T = df[(df["PRODUCT_CODE"] == "T_31")]

    df_test_T = pd.DataFrame()
    T_cols =t_col

    for c, col in enumerate(T_cols):
        df_test_T[col] = df_T[col]

    pred_A = data_A[0].predict(df_test_A)
    pred_T = data_T[0].predict(df_test_T)

    # Code A, T 순서 원래대로 변경
    A_index = df[df["PRODUCT_CODE"] == "A_31"].index
    T_index = df[(df["PRODUCT_CODE"] == "T_31")].index
    O_index = df[(df["PRODUCT_CODE"] == "O_31")].index

    A_add_df = pd.DataFrame()
    A_add_df["index"] = A_index
    A_add_df["predict"] = pred_A

    T_add_df = pd.DataFrame()
    T_add_df["index"] = T_index
    T_add_df["predict"] = pred_T

    O_add_df = pd.DataFrame()
    O_add_df["index"] = O_index
    O_add_df["predict"] = pred_O



    predict_data = pd.concat([A_add_df, T_add_df, O_add_df])
    predict_data = predict_data.sort_values(by=["index"], ascending=[True])

    # submit
    # output_dir = "Result/"
    # if not os.path.exists(output_dir):
    #     os.makedirs(output_dir)

    submit = pd.read_csv(data_path + 'sample_submission.csv')
    submit["Y_Class"] = predict_data["predict"].values
    print(submit["Y_Class"])
    submit.to_csv(data_path + "sola_submission5.csv", index=False)

predict_testset()


[[11  5  1]
 [ 5 29  5]
 [ 1  2 16]]
A_정답률:0.747
!!
     X_1    X_2   X_5   X_7   X_8   X_9   X_11   X_12   X_13  X_15  ...  \
22   2.0  102.0  11.0  45.0  10.0  31.0  514.4  522.1  505.8   0.0  ...   
23   2.0  102.0  11.0  45.0  10.0  31.0  514.6  522.6  506.0   0.0  ...   
25   2.0   97.0  11.0  45.0  10.0  31.0  517.4  526.0  508.9   0.0  ...   
26   2.0   95.0  10.0  54.0  10.0  52.0  477.4  482.4  470.2   0.0  ...   
29   2.0  100.0  11.0  45.0  10.0  31.0  516.5  526.0  507.5   0.0  ...   
..   ...    ...   ...   ...   ...   ...    ...    ...    ...   ...  ...   
589  1.0   94.0  10.0  50.0  10.0  52.0  465.5  470.2  456.1   0.0  ...   
590  1.0   89.0  10.0  50.0  10.0  52.0  466.0  470.5  458.0   0.0  ...   
591  1.0   87.0  10.0  50.0  10.0  52.0  465.9  470.5  456.8   0.0  ...   
592  2.0   98.0  10.0  45.0  10.0  31.0  505.5  510.6  498.8   0.0  ...   
593  2.0   95.0  10.0  50.0  10.0  52.0  468.9  473.9  459.2   0.0  ...   

     X_774  X_775  X_882     X_883  X_884  X_88

/var/folders/js/792bsrvx24lcg4nrdgy7k5br0000gn/T/ipykernel_15259/4133655424.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test_A[col] = df_A[col]
/var/folders/js/792bsrvx24lcg4nrdgy7k5br0000gn/T/ipykernel_15259/4133655424.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test_A[col] = df_A[col]
/var/folders/js/792bsrvx24lcg4nrdgy7k5br0000gn/T/ipykernel_15259/4133655424.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfor

0      1
1      1
2      1
3      1
4      1
      ..
305    1
306    1
307    1
308    1
309    1
Name: Y_Class, Length: 310, dtype: int64


/var/folders/js/792bsrvx24lcg4nrdgy7k5br0000gn/T/ipykernel_15259/4133655424.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test_A[col] = df_A[col]
/var/folders/js/792bsrvx24lcg4nrdgy7k5br0000gn/T/ipykernel_15259/4133655424.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test_A[col] = df_A[col]
/var/folders/js/792bsrvx24lcg4nrdgy7k5br0000gn/T/ipykernel_15259/4133655424.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfor

sola_submission2
상품코드별로 A랑 T랑 O랑 나눠서 했는데 O에는 전부 1만 넣었고,,ㅎㅎ T는 널값 컬럼 다 제거함..!
정확도 : 0.6552052476

In [122]:
from sklearn.ensemble import RandomForestClassifier

seperate_code()


def data_modeling_T():
    data_path = '/Users/sola/Downloads/open/'
    df = pd.read_csv(data_path + 'train.csv')
    # data_T = df[(df["PRODUCT_CODE"] == "T_31") | (df["PRODUCT_CODE"] == "O_31")]
    data_T = df[(df["PRODUCT_CODE"] == "O_31")]

    lists = []
    for a in make_col()[2]:
        lists.append(int(a[2:]))
    lists.sort()

    cols = []  # 사용할 columns
    for l in lists:
        cols.append(f"X_{l}")

    T_y = data_T["Y_Class"]
    T_X = data_T[cols]

    X_train, X_test, y_train, y_test = train_test_split(T_X, T_y, test_size=0.3, random_state=42)

    model_T = lgb.LGBMClassifier(random_state=42)
    model_T.fit(X_train, y_train)
    prediction = model_T.predict(X_test)

    #특성중요도 찾기
    feat_labels = X_train.columns
    importances = model_T.feature_importances_

    # argsort : 리스트 인덱스의 정렬됐을 때의 인덱스 값 반환, [::-1] : 뒤집기
    indices = np.argsort(importances)[::-1]
    a = []
    for i in range(X_train.shape[1]):
        if importances[indices[i]] > 1:
            a.append(feat_labels[indices[i]])

    print(confusion_matrix(prediction, y_test))
    print(f"T_정답률:{accuracy_score(prediction, y_test):.3f}")
    print(a)
    return model_T, cols


data_modeling_T()

Index(['X_1', 'X_2', 'X_7', 'X_8', 'X_11', 'X_12', 'X_13', 'X_15', 'X_16',
       'X_18',
       ...
       'X_897', 'X_898', 'X_899', 'X_900', 'X_901', 'X_902', 'X_903', 'X_904',
       'X_905', 'X_932'],
      dtype='object', length=341)
[[1 1]
 [0 0]]
T_정답률:0.500
[]


(LGBMClassifier(random_state=42),
 ['X_1',
  'X_2',
  'X_11',
  'X_12',
  'X_13',
  'X_16',
  'X_18',
  'X_20',
  'X_21',
  'X_22',
  'X_39',
  'X_40',
  'X_41',
  'X_42',
  'X_43',
  'X_44',
  'X_46',
  'X_47',
  'X_48',
  'X_49',
  'X_50',
  'X_51',
  'X_52',
  'X_53',
  'X_54',
  'X_55',
  'X_56',
  'X_57',
  'X_58',
  'X_59',
  'X_60',
  'X_61',
  'X_62',
  'X_63',
  'X_64',
  'X_65',
  'X_66',
  'X_90',
  'X_120',
  'X_121',
  'X_123',
  'X_400',
  'X_401',
  'X_402',
  'X_403',
  'X_404',
  'X_405',
  'X_442',
  'X_443',
  'X_444',
  'X_446',
  'X_447',
  'X_448',
  'X_450',
  'X_451',
  'X_452',
  'X_453',
  'X_454',
  'X_456',
  'X_457',
  'X_458',
  'X_460',
  'X_461',
  'X_462',
  'X_463',
  'X_464',
  'X_468',
  'X_470',
  'X_471',
  'X_536',
  'X_537',
  'X_538',
  'X_539',
  'X_542',
  'X_543',
  'X_544',
  'X_545',
  'X_559',
  'X_560',
  'X_562',
  'X_563',
  'X_564',
  'X_565',
  'X_566',
  'X_651',
  'X_698',
  'X_753',
  'X_754',
  'X_755',
  'X_756',
  'X_757',
  'X_